In [1]:
get_ipython().system(u'pip install https://github.com/HIPS/autograd/archive/master.zip')

     | 4.5MB 67.9MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-desfj_tv/wheels/1d/cf/71/e88cbaeb7af1fdd864ada20886075a6ffbc935d3b4eaf06a9a
Successfully built autograd


In [2]:
get_ipython().system(u'pip install -U tensorly')

    100% |████████████████████████████████| 81kB 3.9MB/s ta 0:00:011


In [3]:
import numpy as np
import autograd.numpy as anp
from autograd import jacobian
from autograd.scipy.misc import logsumexp
from autograd.scipy.signal import convolve
from scipy.linalg import norm

def softmax(x):    return anp.exp(x - logsumexp(x))
def softplus(x):   return anp.logaddexp(0., x)
def sigmoid(x):    return anp.reciprocal(anp.exp(softplus(-x)))
def flip(x):       return x[::-1]

In [4]:
dim = 6

In [5]:
def function_with_conv(x):
    a = flip(x)
    b = anp.sin(x)
    c = a/b
    d = np.convolve(c, np.array([1,-2,1]), mode='same') # autograd: Mode 'same' not yet implemented
    e = 1./softmax(d)
    return anp.inner(e,e)

In [6]:
def f1(x):
    a = flip(x)
    b = anp.sin(x)
    return a / b

def conv(x):
    return np.convolve(x, np.array([1, -2, 1]), mode='same')


def f2(x):
    e = 1. / softmax(x)
    return anp.inner(e, e)


def vjpForConvolve(v,x):
    r=np.zeros(len(x))
    r[0]=v[0]*(-2)+v[1]*1
    r[len(x)-1]=v[len(v)-2]*1+v[len(v)-1]*(-2)

    for i in range(len(x)-2):
        r[i+1]=v[i]*1+v[i+1]*(-2)+v[i+2]*1
    return r

def gradient(x):
    f1x=f1(x)
    f2j=jacobian(f2)(conv(f1x))
    f1j=jacobian(f1)(x)
    return (vjpForConvolve(f2j,f1x)).dot(f1j)

In [7]:
def approx_grad(f, x0):
    epsilon = 1e-5
    dim = len(x0)
    grad = np.zeros((dim,)).T
    ei   = np.zeros((dim,)).T
    for k in range(dim):
        ei[k] = epsilon
        grad[k] = (f(x0 + ei) - f(x0 - ei))/(2.*epsilon)
        ei[k] = 0.0
    return grad

def check_gradient(f, gradf, x0):
    num_grad = approx_grad(f, x0);
    anal_grad = gradf(x0);
    diff = norm(num_grad - anal_grad)/norm(num_grad + anal_grad)  #отношение норм разницы к сумме градиентов  
    return diff


In [8]:
x = np.random.randn(dim,)
check_gradient(function_with_conv, gradient, x)


2.29142397809056e-06